In [63]:
from lightfm_recomender import LightfmRecomender
import os
%load_ext autoreload
%autoreload 2
new_directory = '/Users/vasevooo/projects/GameBuddy'
os.chdir(new_directory)
import pandas as pd 
import requests
import json
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
from lightfm import LightFM
from scipy.sparse import csr_matrix, save_npz, load_npz

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### обновление предобработанных данных для модели 

In [159]:
ru_ids = pd.read_csv('data/final_gametime_df.csv')
transformed_data = []

for index, row in ru_ids.iterrows():
    user_id = row[-1]  
    
    for game_ids, ranking in row.items():
        if game_ids != 'steamid':

            transformed_data.append([user_id, game_ids, ranking])

transformed_df = pd.DataFrame(transformed_data, columns=['user_id', 'game_id', 'ranking']).astype(int)
transformed_df.head(2)

,user_id,game_id,ranking
0,76561198256283584,10,0
1,76561198256283584,20,0


In [160]:
scaler = MinMaxScaler(feature_range=(0, 100))

transformed_df['ranking'] = transformed_df.groupby('game_id')['ranking'].transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten())
transformed_df['ranking'] = pd.to_numeric(transformed_df['ranking'])

In [161]:
## сохранение df по юзерам для дальнейшего использования

transformed_df.to_csv('data/for_models/users_games_df.csv', index=False)

In [162]:
games = pd.read_csv('data/game_info_with_scores.csv')
item_dict ={}
games = games[['appid', 'Name']].sort_values('appid').reset_index()
games['appid'] = games['appid'].astype(int)

for i in range(games.shape[0]):
    item_dict[int(games.loc[i,'appid'])] = games.loc[i,'Name']

In [163]:
## сохранение словаря по играм для дальнейшего использования

with open("data/for_models/item_dict.json", "w") as outfile:
    json.dump(item_dict, outfile, indent=4)

### Тест класса LigthFM Recomender

In [173]:
item_dict_path = 'data/for_models/item_dict.json'
users_games_df_path = 'data/for_models/users_games_df.csv'

lfr = LightfmRecomender(users_games_df_path, item_dict_path)

In [174]:
steamid = 76561197992623144

In [175]:
new_user = lfr.get_user_games(steamid)

In [176]:
matrix, df = lfr.get_csr_matrix_for_all_users(new_user)

In [177]:
model = lfr.fit_model(matrix, epochs=10)

In [178]:
lfr.sample_recommendation_user(model, df, steamid, threshold = 0, nrec_items = 5)

User: 76561197992623144
Recommended Items:
1- Total War: NAPOLEON – Definitive Edition
2- Valheim
3- Valiant Hearts: The Great War™ / Soldats Inconnus : Mémoires de la Grande Guerre™
4- Age of Mythology: Extended Edition
5- Lord of the Rings: War in the North

 Known Likes:
1- V Rising
2- Orcs Must Die! 3
3- Sea of Thieves 2023 Edition
4- Apex Legends™
5- Crusader Kings III
6- Fall Guys
7- Cyberpunk 2077
8- Going Medieval
9- Among Us
10- Back 4 Blood
11- Age of Empires II: Definitive Edition
12- Gloomhaven
13- Blasphemous
14- Disco Elysium - The Final Cut
15- Graveyard Keeper
16- Total War: WARHAMMER II
17- Warhammer 40,000: Gladius - Relics of War
18- The Banner Saga 3
19- Wallpaper Engine
20- Firewatch
21- Kingdom Come: Deliverance
22- Total War: WARHAMMER
23- Clicker Heroes
24- Frostpunk
25- Tabletop Simulator
26- This War of Mine
27- The Banner Saga 2
28- Mount & Blade II: Bannerlord
29- Banished
30- Assassin’s Creed® IV Black Flag™
31- Papers, Please
32- The Banner Saga
33- Baldur